In [1]:
import PySimpleGUI as sg
import serial
import sys
import numpy as np
import os
from simple_pid import PID
import minimalmodbus
import pandas as pd
import threading
import datetime


---

In [2]:
def serial_ports():
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Платформа не поддерживается')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result


In [3]:
class ValveCtrlSystem():
    def __init__(self):
        
        # инициализация ПИД регуляторов для клапанов
        
        self.target = 60
        
        self.pid_pos_1 = PID(1500, 50, 1, setpoint=self.target)
        self.pid_pos_1.output_limits = (0, 11000) 
        self.pid_pos_2 = PID(1500, 50, 1, setpoint=self.target)
        self.pid_pos_2.output_limits = (0, 11000) 
        
    def set_target(self, target):
        
        # установка уставки
        
        self.target = target
        self.pid_pos_1.setpoint = target
        self.pid_pos_2.setpoint = target
        
    def ctrl_signal(self, current_value):
        
        # расчет и выдача управляющего сигнала
        
        pos_1_ctrl_signal = int(self.pid_pos_1(current_value))
        pos_2_ctrl_signal = int(11000 - self.pid_pos_2(current_value))
        return pos_1_ctrl_signal, pos_2_ctrl_signal

In [4]:
class HumidityGenerator():
    def __init__(self, COM_PORT):
        
        try:
            
            self.instrument = minimalmodbus.Instrument(COM_PORT, 0xD7)
        
            self.instrument.serial.baudrate = 19200
            self.instrument.serial.bytesize = 8
            self.instrument.serial.stopbits = 1
            
        except Exception as e: 
            print('Проверьте работоспособность COM-порта и выполните инициализацию класса снова. ' + 'Текст ошибки: ' + str(e))
        
        self.loggs = []
        
        # влажность
        self.humidity = 0.0
        
        # расход
        self.g_1 = 0.0
        self.g_2 = 0.0
        self.g_tot = 0.0
        
        # температура
        self.temp = 0.0
        
        # время    
        self.t = 0.0
        
        # для отправки
        
        self.pos_1 = 0
        self.pos_2 = 0
        self.t_discr = 0
        self.t_work = 0
        
    def update(self, recived_data, target):
        
        # обновление значений 
        
        self.humidity = recived_data[0]

        self.g_1 = recived_data[2]

        self.g_2 = recived_data[3]

        self.g_tot = recived_data[2] + recived_data[3]

        self.temp = recived_data[1]

        self.t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # запись строки в логи
        
        self.loggs.append([target,
                           self.humidity, 
                           self.g_1,
                           self.g_2, 
                           self.g_tot, 
                           self.temp, 
                           self.t,
                           self.pos_1, 
                           self.pos_2, 
                           self.t_discr,
                           self.t_work])
        
    def get_params(self):
        
        # получение значений с генератора
        
        try:
            recived_data = self.instrument.read_registers(0, 4, functioncode=3)
            return recived_data
        except:
            return [0, 0, 0, 0]
        
    def send_params(self, send_data):
        
        # отправка значений в генератор
        
        self.pos_1 = send_data[0]
        self.pos_2 = send_data[1]
        self.t_discr = send_data[2]
        self.t_work = send_data[3]
        
        self.instrument.write_registers(0, send_data)
        
    def write_loggs(self):
        
        dir = os.path.abspath(os.curdir)
        
        name = 'loggs_'+ str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
        
        # запись логов в эксель
        
        df = pd.DataFrame(data=self.loggs, columns = ['Влажность-уставка, %',
                                                      'Влажность, %',
                                                      'Расход 1, мл',
                                                      'Расход 2, мл',
                                                      'Сумма расх., мл',
                                                      'Температура, гр.Цел.',
                                                      'Время',
                                                      'Клапан 1, полож', 
                                                      'Клапан 2, полож', 
                                                      'Время дискр, мс', 
                                                      'Время вкл, мс'])
        df.to_excel(dir + '/' + name + '.xlsx')
        return name + '.xlsx'

In [5]:
def daemon(humidity_generator, valve_ctrl_system, delay, event_thr):
    while True:
        # получение данных
        recived_data = humidity_generator.get_params()
        current_humidity = recived_data[0]
    
        # обновление данных в классе HumidityGenerator
    
        humidity_generator.update(recived_data, valve_ctrl_system.target)
    
        # расчет управляющего сигнала
    
        pos_1_ctrl_signal, pos_2_ctrl_signal = valve_ctrl_system.ctrl_signal(current_humidity)
    
        # отправка данных
    
        send_data = [pos_1_ctrl_signal, pos_2_ctrl_signal, 10000, 0]
    
        humidity_generator.send_params(send_data)
        
        print(humidity_generator.t,
              ' Влажность, %: ', current_humidity,
              ' Уставка, %: ', valve_ctrl_system.target)

    
        event_thr.wait(delay)
        if event_thr.is_set():
            break

---

In [6]:
def choose_com_port():
    
    global COM_PORT
    combo=sg.Combo(serial_ports(),default_value=None,key='board')
    layout = [[sg.Text("Выберите COM-порт с устройством")],
                       [combo, sg.Button('Выбрать')]]
    window = sg.Window('Выбор COM-порта', layout, size=(290,90))
    while True:
        event, values = window.read()
        if event == sg.WINDOW_CLOSED:
            break
        if event == 'Выбрать': 
            keys_entered = ''
            COM_PORT = combo.get()
            if COM_PORT == '':
                error_empty_com_port()
                window.close()
                break
            else:
                window.close()
                break
    return COM_PORT

In [7]:
def error_empty_com_port():
    layout = [[sg.Text("Выбран пустой COM-порт!")],
              [sg.Button('Назад')]]
    window = sg.Window('Ошибка', layout, size=(290,90))
    while True:
        event, values = window.read()
        if event == sg.WINDOW_CLOSED or event == 'Назад':
            window.close()
            choose_com_port()
            break

In [8]:
def main_window(COM_PORT):
    
    
    STEP_SIZE = 1  
    SAMPLES = 100 
    SAMPLE_MAX = 100 
    CANVAS_SIZE = (500, 300)
    LABEL_SIZE = (1000, 20)


    pt_values = []
    pt_times = []
    for i in range(SAMPLES+1): 
        pt_values.append("")
        pt_times.append("")
    
    

    graph = sg.Graph(CANVAS_SIZE, (0, 0), (SAMPLES, SAMPLE_MAX),
          background_color='black', key='graph')
    
    
    
    
    valve_ctrl_system = ValveCtrlSystem()
    humidity_generator = HumidityGenerator(COM_PORT)
    event_thr = threading.Event()
    event_thr.clear()
    thr1 = threading.Thread(target = daemon, args=(humidity_generator, valve_ctrl_system, 1, event_thr)).start()


    layout = [[sg.Text("Влажность:"), sg.Text('     ',  key='output'), sg.Text("%")],
              [sg.Text("Уставка:"), sg.I('', key='target', size=(15,1),pad=(10,10)), sg.Text("%"), sg.Button('Задать')],
              [graph, sg.Output(size=(80,18))],  
              [sg.Button(button_text="Остановить и выйти", button_color=('white', 'red')),
               sg.Button(button_text="Записать логи", button_color=('white', 'green'))]]
    window = sg.Window('Генератор влажности', layout)
    
    graph = window['graph']
    output = window['output']
    
    
    
    
    
    i = 0
    prev_x, prev_y = 0, 0
    prev_target = valve_ctrl_system.target

    while True: 
  
        event, values = window.read(timeout=1000)
        if event == 'Остановить и выйти' or sg.WINDOW_CLOSED: 
            event_thr.set()
            break
        if event == 'Записать логи': 
            name  = humidity_generator.write_loggs()
            print(humidity_generator.t, f'Логи записаны в файл с именем {name} в папку с программой')
            
        if event == 'Задать':
            target = valve_ctrl_system.target
            humidity = humidity_generator.humidity
            try:
                print(str(humidity_generator.t) +' Задана уставка:', int(values['target']), '%')
                valve_ctrl_system.set_target(int(values['target']))
            except:
                continue


        data_pt = humidity_generator.humidity
        now_time = humidity_generator.t
        pt_values[i] = data_pt       
        pt_times[i] = str(now_time) 

        window['output'].update(data_pt)
        
        if data_pt > SAMPLE_MAX:
            data_pt = SAMPLE_MAX
        new_x, new_y = i, data_pt
        new_target = valve_ctrl_system.target

        if i >= SAMPLES:
            graph.move(-STEP_SIZE, 0)
            prev_x = prev_x - STEP_SIZE
            for i in range(SAMPLES):
                pt_values[i] = pt_values[i+1]
                pt_times[i] = pt_times[i+1]
                    
        graph.draw_line((prev_x, prev_target), (new_x, new_target), color='blue')
        graph.DrawPoint((new_x, new_y), size=0.3, color='red')
        
        prev_x, prev_y = new_x, new_y
        prev_target = new_target
        i += STEP_SIZE if i < SAMPLES else 0
        
    window.close()

In [9]:
main_window(choose_com_port())